In [16]:
import pandas as pd
import datetime as dt

from load import Load
from transform import Transform
from save import save
from download import DataGetter
from pathlib import Path


In [17]:
diesel_sheet_name = 'DPCache_m3_2'
oilder_sheet_name = 'DPCache_m3'

sheet_file = Path('vendas.ods')


In [18]:
df1b = pd.read_excel(sheet_file, oilder_sheet_name)


In [19]:
def transform(table: pd.DataFrame) -> pd.DataFrame:

    # unit of every product (hardcoded)
    unit = 'm3'

    # listing mapping brazilian months to integers
    months = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
    months_to_int_d = {m: i for m, i in zip(months, range(1, 12 + 1))}
    months_to_int = lambda x: months_to_int_d[x]

    # transforming the data
    table = table.melt(id_vars=['ANO', 'COMBUSTÍVEL', 'ESTADO'],
                       value_vars=months, var_name='MONTH',
                       value_name='volume')
    table['month_int'] = table.apply(lambda x: months_to_int(x['MONTH']), axis=1)
    table['year_month'] = table.apply(lambda x: dt.datetime(x['ANO'], x['month_int'], 1), axis=1)
    table.rename({'COMBUSTÍVEL': 'product', 'ESTADO': 'uf'}, inplace=True, axis=1)
    table['unit'] = unit
    table['created_at'] = pd.Timestamp.now()
    table = table[['year_month', 'uf', 'product', 'unit', 'volume', 'created_at']]

    return table



In [20]:
df1 = transform(df1b)

QUEROSENE DE AVIAÇÃO (m3)	2020	REGIÃO NORTE	RONDÔNIA

COMBUSTÍVEL	ANO	REGIÃO	ESTADO


In [29]:
df1[(df1['product']=='QUEROSENE DE AVIAÇÃO (m3)') & (df1['uf']=='RONDÔNIA') & (df1['year_month']==dt.datetime(2020, 3, 1))].dtypes

year_month    datetime64[ns]
uf                    object
product               object
unit                  object
volume               float64
created_at    datetime64[ns]
dtype: object

In [30]:
pd_df = df1.where((pd.notnull(df1)), None)

In [33]:
pd_df[(pd_df['product']=='QUEROSENE DE AVIAÇÃO (m3)') & (pd_df['uf']=='RONDÔNIA') & (pd_df['year_month']==dt.datetime(2020, 3, 1))]

,year_month,uf,product,unit,volume,created_at
11313,2020-03-01,RONDÔNIA,QUEROSENE DE AVIAÇÃO (m3),m3,0.0,2023-04-09 17:07:31.555311


In [32]:
pd_df = df1.fillna(0)